## Naive Bayes y Clasificación de Texto

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

En esta ayudantía trabajaremos con el modelo Naive Bayes y aprovecharemos que es muy usado para clasificación de texto para dar una introducción (muy superficial) a cómo pueden trabajar con este tipo de datos.
Para esto, trabajaremos con un dataset de SMS que han sido clasificados como Spam o Ham (No Spam). La base fue descargada de: [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/228/sms+spam+collection).

Primero veamos más en detalle el modelo:


![Naive Bayes Model](https://afit-r.github.io/public/images/analytics/naive_bayes/naive_bayes_icon.png)

El modelo se basa en el teorema de Bayes para estimar la probabilidad de que una observación corresponda a una cierta clase, dadas las características de esa observación.

El denominador de la fracción de la derecha en realidad no nos importa, porque nuestro $x$ ya está dado, por lo tanto, es constante, así que nos fijaremos solo en el numerador.

La probabilidad a priori de que un observación corresponda a cierta clase $P(c)$, el modelo la estima a partir de la presencia de las clases en nuestro dataset de entrenamiento (aunque esto es un parámetro que se puede entregar por fuera o, simplmente, usar una distribución uniforme).

La verosimilitud, $P(x|c)$, el modelo la estima calculando la presencia de cada feature $x$ para cada clase $c$. Por lo tanto, esta es la parte en que el modelo aprende realmente de las features que le entregamos.

Vamos con la aplicación práctica:

In [ ]:
# Leemos nuestros datos desde Google Drive
import requests
url = "https://drive.google.com/uc?id=13I_Te6decSUBcgzqxwK6qUaFwCtW9nou"
data = requests.get(url)
data.text

In [ ]:
# Nuestros datos vienen en un solo string, donde cada línea está separada por un new line character (\n)
# Y dentro de cada línea, tenemos primero la etiqueta y luego el mensaje de texto, separados por un tab (\t)
# Usando esta estructura, vamos a procesar nuestros datos y separarlos en dos listas
list_txt = data.text.split('\n')
txts = []
labels = []
for el in list_txt:
    splitted = el.split('\t')
    txts.append(splitted[1])
    labels.append(splitted[0])

In [ ]:
list_txt[0]

In [ ]:
print(txts[:5])

In [ ]:
print(labels[:5])

In [ ]:
len(txts)

In [ ]:
pd.Series(labels).value_counts()

In [ ]:
pd.Series(labels).value_counts("%")

Ya teniendo nuestros textos listos para ser trabajados, aquí podríamos aplicar muchas técnicas de preprocesamiento:
- Estandarización de palabras (para corregir errores de ortografía, por ejemplo)
- Reemplazo de abreviaciones
- Identificación de palabras que no nos ayudan a clasificar el texto (por ejemplo, nombres propios)
- Etc.

Pero como esta es solo una breve introducción al procesamiento de textos, no aplicaremos ninguna de ellas e iremos directo al splitting y preparación de los datos para entregarselos a nuestros modelos:

In [ ]:
# Separación en train y test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(txts, labels, test_size=0.2, random_state=0)

Como saben, nuestros modelos siempre esperan features numéricas, por lo tanto no podemos pasarles directamente los textos de nuestro training set.
Existen varias técnicas para transformar textos en representaciones numéricas y vamos a ver primero la vectorización por conteo.
Lo que vamos a hacer es identificar todas las palabras que están en nuestro training set (corpus) y contaremos las veces que aparecen en cada una de nuestras observaciones para representarlas:

In [ ]:
# Importamos nuestro vectorizador, lo ajustamos con SOLO el training set y transformamos tanto nuestro training como nuestro test set:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Tenemos 7720 palabras distintas en nuestro
len(vectorizer.get_feature_names_out())

Miremos más en detalle cómo se transformó el primer mensaje de texto:

In [ ]:
X_train[0]

In [ ]:
# Veamos en qué indices nuestra matriz quedó con valores mayores a 0:
ix = np.where(X_train_vec.toarray()[0]>0)
ix

In [ ]:
# Ahora veamos qué valores están en esas posiciones
X_train_vec.toarray()[0][ix]

In [ ]:
# Veamos a qué palabras de nuestro universo de palabras corresponden esas posiciones
vectorizer.get_feature_names_out()[ix]

Ahora, vamos a probar dos modelos distintos para comparar sus performances. 

En realidad, son el mismo modelo de Naive Bayes pero lo que cambia es el kernel que usan.

El kernel es la distribución de probabilidad que asume el modelo por detrás para calcular las verosimilitudes.

In [ ]:
# Partiremos con un kernel Gaussiano (Distribución Normal)
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(X_train_vec.toarray(), y_train).predict(X_test_vec.toarray())

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Al modelo con el kernel Gaussiano le va bastante bien.

Probemos ahora con un kernel Multinomial:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
y_pred = mnb.fit(X_train_vec.toarray(), y_train).predict(X_test_vec.toarray())

In [ ]:
print(classification_report(y_test, y_pred))

A este modelo le va mucho mejor! Tiene que ver con el kernel que usamos:

El kernel Gaussiano es más apropiado para features con valores continuos, mientras que el Multinomial lo es para features discretas, como las que tenemos en nuestro training set.

Veamos algunos mensajes en los que se equivocó:

In [ ]:
for i in range(5):
    ix = np.where(y_test != y_pred)[0][i]
    print(X_test[ix], y_test[ix], y_pred[ix])

Probemos qué pasaría si el modelo no asumiese que hay más mensajes de ham que de spam, es decir, modifiquemos las distribuciones a priori:

In [ ]:
mnb_uniform = MultinomialNB(fit_prior=False)
y_pred = mnb_uniform.fit(X_train_vec.toarray(), y_train).predict(X_test_vec.toarray())
print(classification_report(y_test, y_pred))

In [ ]:
np.exp(mnb.class_log_prior_)

In [ ]:
np.exp(mnb_uniform.class_log_prior_)

#### Probemos ahora con otro tipo de vectorización: TF-IDF
TF-IDF significa Term Frequency times Inverse Document Frequency

Term Frequency es simplemente la frecuencia (en porcentaje) de aparición de cada palabra en cada uno de nuestros textos. Entonces, va a ser igual a nuestra vectorización por cuenta, pero cada valor dividido por la cantidad total de palabras en cada texto.

Inverse Document Frequency nos va a permitir darle menos peso a las palabras que aparecen en muchos texto. Se calcula como el logaritmo de la división entre el número total de textos y la cantidad de textos donde aparece una palabra en específico.

![tf idf](https://qph.cf2.quoracdn.net/main-qimg-60a54c42850675139e2899634d3a669c)

In [ ]:
# Importamos nuestro vectorizador, lo ajustamos y transformamos ambos sets de datos

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_vec = tfidf_vectorizer.fit_transform(X_train)
X_test_vec = tfidf_vectorizer.transform(X_test)

Veamos cómo quedaron nuestras features ahora:

In [ ]:
X_train[10]

In [ ]:
# Veamos en qué indices nuestra matriz quedó con valores mayores a 0:
ix = np.where(X_train_vec.toarray()[10]>0)
ix

In [ ]:
# Ahora veamos qué valores están en esas posiciones
X_train_vec.toarray()[10][ix]

In [ ]:
# Veamos a qué palabras de nuestro universo de palabras corresponden esas posiciones
tfidf_vectorizer.get_feature_names_out()[ix]

In [ ]:
print(list(zip(tfidf_vectorizer.get_feature_names_out()[ix], X_train_vec.toarray()[10][ix])))

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train_vec.toarray(), y_train).predict(X_test_vec.toarray())
print(classification_report(y_test, y_pred))

In [ ]:
mnb = MultinomialNB()
y_pred = mnb.fit(X_train_vec.toarray(), y_train).predict(X_test_vec.toarray())
print(classification_report(y_test, y_pred))